In [1]:
import pandas as pd
import os

In [2]:
filepath = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志'
logart = 'logart_all_toc_20190728.csv'

read_logart = '{0}/{1}'.format(filepath, logart)

In [17]:
workDf = pd.read_csv(read_logart)
workDf = workDf.fillna('0')
workDf.shape, workDf.head()

((365771, 6),    bookId bookName sectionName  level  startPage  endPage
 0  '00201     甕安縣志          卷首      1          1       23
 1  '00201     甕安縣志           序      2          1       10
 2  '00201     甕安縣志    甕安縣志纂輯銜名      2         11       14
 3  '00201     甕安縣志      甕安縣志目次      2         15       21
 4  '00201     甕安縣志           圖      2         22       23)

# Synonym

In [4]:
synonymFile = 'fangzhi_synonym.owl'

read_synonym = '{0}/{1}'.format(filepath, synonymFile)

In [5]:
import json

with open(read_synonym , 'r') as reader:
    jf = json.loads(reader.read())

catagory = {}

# Read all nodes in first layer
for obj in jf:
    tmpId = obj['@id']
    pos = tmpId.rfind('#')
    node = tmpId[pos+1:len(tmpId)]
    
    if '(' not in node:
        continue
        
    node = node.replace('(', '').replace(')', '')
    catagory[node] = {'parent':[], 'synonym':[node]}

catagory['未歸類'] = {'parent':[], 'synonym':[]}

for obj in jf:
    if 'http://www.w3.org/2000/01/rdf-schema#subClassOf' not in obj:
        continue
        
    tmpId = obj['@id']
    pos = tmpId.rfind('#')
    subClass = tmpId[pos+1:len(tmpId)]
    subClass = subClass.replace('(', '').replace(')', '')

    tmpSubClassOf = obj['http://www.w3.org/2000/01/rdf-schema#subClassOf'][0]['@id']
    pos = tmpSubClassOf.rfind('#')
    parent = tmpSubClassOf[pos+1:len(tmpSubClassOf)]
    parent = parent.replace('(', '').replace(')', '')
    
    if parent not in catagory:
        print("ERROR!!", parent, subClass)
        
    if subClass not in catagory[parent]['synonym']:
        catagory[parent]['synonym'].append(subClass)

print(catagory)

{'事紀': {'parent': [], 'synonym': ['事紀', '事記', '紀事', '記事', '邑紀事']}, '亭榭': {'parent': [], 'synonym': ['亭榭']}, '人物': {'parent': [], 'synonym': ['人物']}, '仙釋': {'parent': [], 'synonym': ['仙釋']}, '倉儲': {'parent': [], 'synonym': ['倉儲', '倉貯', '積儲', '積貯']}, '公署': {'parent': [], 'synonym': ['公署', '廨署', '文署', '署廨']}, '兵餉': {'parent': [], 'synonym': ['兵餉']}, '冢墓': {'parent': [], 'synonym': ['冢墓', '丘墓', '塚墓', '墓冢', '墓塜', '墳墓', '邱墓']}, '叢談': {'parent': [], 'synonym': ['叢談', '叢譚']}, '名勝': {'parent': [], 'synonym': ['名勝']}, '喪葬': {'parent': [], 'synonym': ['喪葬']}, '圖經': {'parent': [], 'synonym': ['圖經']}, '坊巷': {'parent': [], 'synonym': ['坊巷']}, '域市': {'parent': [], 'synonym': ['域市', '坊都', '城市', '都坊']}, '堡村': {'parent': [], 'synonym': ['堡村']}, '墟市': {'parent': [], 'synonym': ['墟市', '圩埠', '圩市', '圩集', '墟埠', '墟場', '墟屯', '墟集', '集市']}, '壇廟': {'parent': [], 'synonym': ['壇廟', '廟祠', '祠廟']}, '存留': {'parent': [], 'synonym': ['存留']}, '學宮': {'parent': [], 'synonym': ['學宮']}, '學校': {'parent': [], 'synonym': ['學校']}

# Catagory

In [6]:
catagoryFile = 'fangzhi_category.owl'

read_catagory = '{0}/{1}'.format(filepath, catagoryFile)

In [7]:
import json

# catagory = {}
with open(read_catagory , 'r') as reader:
    jf = json.loads(reader.read())
      
def findBelong(INclass, obj):
    if INclass in obj:
        return INclass

    for key in obj:
        if INclass in obj[key]['synonym']:
            return key
    return ""
    
for obj in jf:
    if 'http://www.w3.org/2000/01/rdf-schema#subClassOf' not in obj:
        continue
        
    tmpId = obj['@id']
    pos = tmpId.rfind('#')
    subClass = tmpId[pos+1:len(tmpId)]

    tmpSubClassOf = obj['http://www.w3.org/2000/01/rdf-schema#subClassOf'][0]['@id']
    pos = tmpSubClassOf.rfind('#')
    parent = tmpSubClassOf[pos+1:len(tmpSubClassOf)]
    
    OUTclass = findBelong(subClass, catagory)
    if OUTclass == "未歸類":
        continue
        
    if OUTclass == "":
        catagory[subClass] = {'parent':[parent], 'synonym':[subClass]}
    else:
        catagory[OUTclass]['parent'].append(parent)

print(catagory)

{'事紀': {'parent': ['編年', '藝文'], 'synonym': ['事紀', '事記', '紀事', '記事', '邑紀事']}, '亭榭': {'parent': [], 'synonym': ['亭榭']}, '人物': {'parent': [], 'synonym': ['人物']}, '仙釋': {'parent': [], 'synonym': ['仙釋']}, '倉儲': {'parent': [], 'synonym': ['倉儲', '倉貯', '積儲', '積貯']}, '公署': {'parent': [], 'synonym': ['公署', '廨署', '文署', '署廨']}, '兵餉': {'parent': ['兵防'], 'synonym': ['兵餉']}, '冢墓': {'parent': ['勝蹟'], 'synonym': ['冢墓', '丘墓', '塚墓', '墓冢', '墓塜', '墳墓', '邱墓']}, '叢談': {'parent': [], 'synonym': ['叢談', '叢譚']}, '名勝': {'parent': [], 'synonym': ['名勝']}, '喪葬': {'parent': [], 'synonym': ['喪葬']}, '圖經': {'parent': [], 'synonym': ['圖經']}, '坊巷': {'parent': [], 'synonym': ['坊巷']}, '域市': {'parent': [], 'synonym': ['域市', '坊都', '城市', '都坊']}, '堡村': {'parent': [], 'synonym': ['堡村']}, '墟市': {'parent': [], 'synonym': ['墟市', '圩埠', '圩市', '圩集', '墟埠', '墟場', '墟屯', '墟集', '集市']}, '壇廟': {'parent': [], 'synonym': ['壇廟', '廟祠', '祠廟']}, '存留': {'parent': [], 'synonym': ['存留']}, '學宮': {'parent': [], 'synonym': ['學宮']}, '學校': {'parent': [], 

# find catagory from gatateers

In [45]:
# for idx in range(0, len(workDf)):
gatateerCata = {}

curBookId = ""
curBookName = ""

bookList = {}
bookHierarchy = {}
curLayers = {'1':'', '2':''}
for idx in range(0, len(workDf)):
    row = workDf.loc[idx]

    bookName = row['bookName']
    sectionName = row['sectionName']
    level = row['level']
    bookId = row['bookId']
    
    if curBookId == "":
        curBookId = bookId
        curBookName = bookName

    if curBookId != bookId:
        bookList[curBookName] = {'root':bookHierarchy, 'bookId':curBookId}
        
        curBookId = bookId
        curBookName = bookName
        bookHierarchy = {}
        curLayers['1'] = ''
        curLayers['2'] = ''

    if idx >= len(workDf)-1:
        bookList[bookName] = {'root':bookHierarchy, 'bookId':curBookId}
        break

    if level == 1:
        if sectionName not in bookHierarchy:
            bookHierarchy[sectionName] = {}
        curLayers['1'] = sectionName
    elif level == 2:
        try:
            if curLayers['1'] == '':
                curLayers['1'] = bookName
            if curLayers['1'] not in bookHierarchy:
                bookHierarchy[curLayers['1']] = {}
            if sectionName not in bookHierarchy[curLayers['1']]:
                bookHierarchy[curLayers['1']][sectionName] = {}
            curLayers['2'] = sectionName
        except:
            print("ERROR2:", idx, bookId, bookName, sectionName, level)
            break
    elif level == 3:
        try:
            if curLayers['1'] not in bookHierarchy:
                bookHierarchy[curLayers['1']] = {}
            if curLayers['2'] not in bookHierarchy[curLayers['1']]:
                bookHierarchy[curLayers['1']][curLayers['2']] = {}
            if sectionName not in bookHierarchy[curLayers['1']][curLayers['2']]:
                bookHierarchy[curLayers['1']][curLayers['2']][sectionName] = {}

        except:
            errRow = [idx, bookId, bookName, sectionName, level]
            print("ERROR3:", idx, bookId, bookName, sectionName, level)
            break

print(len(bookList))

2658


In [55]:
# bookName: {'root':object, 'bookId':string}
print(bookList['甕安縣志'])

{'root': {'卷首': {'序': {}, '甕安縣志纂輯銜名': {}, '甕安縣志目次': {}, '圖': {}}, '卷一': {'輿圖': {}}, '卷二': {'疆域：形勝附': {}}, '卷三': {'紀年上': {}, '紀年中': {}, '紀年下': {}}, '卷四': {'城池：村市、關梁、郵傳附': {}}, '卷五': {'公署：壇廟、寺觀、名勝、坊表、古蹟、邱墓附': {}}, '卷六': {'山川：水道考附': {}}, '卷七': {'氣候': {}}, '卷八': {'祥異': {}}, '卷九': {'户口：風俗、方言、苗蠻附': {}}, '卷十': {'職官': {}}, '卷十一': {'學校：學產附': {}}, '卷十二': {'科貢': {}}, '卷十三': {'典禮：褎揚條例附': {}}, '卷十四': {'農業：釐桑、木棉附': {}}, '卷十五': {'田賦：徭役、榷估、經費附': {}}, '卷十六': {'積貯': {}}, '卷十七': {'土官': {}}, '卷十八': {'軍制': {}}, '卷十九': {'列傳': {'名宦': {}, '儒林': {}, '孝友': {}, '賢能：宦蹟附': {}, '武功': {}, '忠節': {}, '行義': {}, '文苑': {}, '隱逸': {}, '寓賢': {}, '耆壽': {}, '節孝：烈婦、孝婦、淑媛、貞女、烈女、義婢附': {}}}, '卷二十': {'藝文上': {'賦': {}, '詩': {}}, '藝文下': {'啟': {}, '議': {}, '書': {}, '序': {}, '記': {}, '論': {}, '辨': {}, '說': {}, '墓志銘': {}, '跋': {}}}, '卷二十一': {'雜記': {}}, '自敘': {}}, 'bookId': "'00201"}


In [71]:
bookObj = bookList['甕安縣志']['root']

def findKeyInObj(key, obj):
#     print(key, obj)
    if key in obj:
#         print("found!!", key)
        return key
    
    for nKey in obj:
        if obj[nKey] != {}:
            res = findKeyInObj(key, obj[nKey])
            if res != '':
                return res
    return ''
    
# for key in bookObj:
key = findKeyInObj('農業', bookObj)
print(key)
# print(bookObj['卷首']['序'] == {})

In [75]:
firstTypeBooks = []
for bookName in bookList:
    for title in bookList[bookName]['root']:
        if '卷一' in title:
            firstTypeBooks.append(bookName)
            break

print(len(firstTypeBooks))

958


In [76]:
firstTypeBooks

['甕安縣志',
 '福鼎縣志',
 '許昌縣志',
 '臨澤縣志',
 '富平縣志稿',
 '新續渭南縣志',
 '武陟縣志',
 '臨榆縣志',
 '南漳縣志',
 '永壽縣志',
 '洪洞縣志',
 '順德縣志',
 '孝豐縣志',
 '海鹽縣志',
 '廣州府志',
 '太谷縣志',
 '陽江志',
 '商水縣志',
 '禹縣志',
 '湄潭縣志',
 '山丹縣志',
 '永泰縣志',
 '芮城縣志',
 '路南縣志',
 '臨晉縣志',
 '交河縣志',
 '永年縣志',
 '鄖西縣志',
 '漢州志',
 '渠縣志',
 '河內縣志',
 '修武縣志',
 '新蔡縣志',
 '梓潼縣志',
 '霸縣新志',
 '南陵縣志',
 '甘州府志',
 '考城縣志',
 '孟縣志',
 '郟縣志',
 '定陶縣志',
 '文水縣志',
 '武邑縣志',
 '賀縣志',
 '林縣志',
 '鎮安府志',
 '陸川縣志',
 '萍鄉縣志',
 '富川縣志',
 '濟源縣志',
 '續武陟縣志',
 '獲嘉縣志',
 '臨潼縣志',
 '百色廳志',
 '昭平縣志',
 '恭城縣志',
 '重修滑縣志',
 '陵縣續志',
 '無錫縣志',
 '鬱林州志',
 '臨桂縣志',
 '清源鄉志',
 '鄢陵縣志',
 '南陽縣志',
 '貴縣志',
 '開平縣志',
 '鄒平縣志',
 '蓋平縣志',
 '錦縣志',
 '遼陽州志',
 '岫嚴志略',
 '薊縣志',
 '口北三廳志',
 '承德府志',
 '赤城縣志',
 '寧夏府志',
 '象州志',
 '隆安縣志',
 '滎經縣志',
 '和林格爾廳志',
 '河北通志稿',
 '會理州志',
 '江安縣志',
 '羅江縣志',
 '滿城縣志略',
 '蒙古志',
 '遷安縣志',
 '瓊東縣志',
 '雙流縣志',
 '望都縣志',
 '新疆志稿',
 '宜賓縣志',
 '烏程縣志',
 '蘭谿縣志',
 '平陽縣志',
 '遂安縣志',
 '台州府志',
 '懷柔縣新志',
 '南宮縣志',
 '唐縣志',
 '西和縣志',
 '崇信縣志',
 '和政縣志',
 '蒲城縣新志',
 '赤溪縣志',
 '高要縣志',
 '莆田縣志',
 '懷集縣志',
 '樂昌縣志',
 '陽山縣志',
 '麻江縣志',
 '無